In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import zipfile

In [12]:
B = list(range(400))
city_to_run = 'Detroit'
day = 365                                   # Equivalent days
starting_day = 0
hour = day * 24
starting_hour = starting_day * 24

li = []
for b in B:
    building_id = b

    # Read resstock results summary folder:
    model_dir = '/home/anph/projects/TES/Data/'
    load_folder = '400_Buildings_EB/' + city_to_run + '/'

    resstock_results = pd.read_csv(model_dir + load_folder + 'results.csv')
    resstock_building_id = building_id + 1

    folder_id = resstock_results.loc[resstock_results['build_existing_model.building_id'] == resstock_building_id, '_id']
    folder_id = folder_id.reset_index()
    folder_id = folder_id.drop(['index'], axis=1)
    folder_id = folder_id.iloc[0,0]

    archive = zipfile.ZipFile(model_dir + load_folder + folder_id + '/' + 'data_point.zip')

    load_data_path = archive.extract('enduse_timeseries.csv', model_dir + load_folder + folder_id + '/')
    load_data_raw = pd.read_csv(load_data_path)
    archive.close()

    # Hourly load
    load_data_annual = load_data_raw['electricity_heating_kwh']
    load_data_annual = load_data_annual.iloc[starting_hour:starting_hour+hour]
    load_data_annual = load_data_annual.reset_index()
    load_data_annual = load_data_annual.drop(['index'], axis=1)
    load_data_annual = load_data_annual.rename(columns={'electricity_heating_kwh':0})

    peakLoad = float(load_data_annual.max())
    df = pd.DataFrame()
    df = df.append({'building id' : int(resstock_building_id), 'Peak Load' : peakLoad}, ignore_index = True)
    li.append(df)

peakLoadAll = pd.concat(li, axis=0)
peakLoadAll.to_excel('/home/anph/projects/TES/Results/Detroit/Compiled/' + 'Peakload.xlsx')


KeyboardInterrupt: 